In [1]:
import numpy as np
from scipy.special import erf
from math import floor, log10

In [2]:
# Segura
d = np.array([0.1545314772435472, 0.5354555247125108, 0.4443376735387033])
a = np.array([3.423766464751332, 0.6234266839478166, 0.1687830644383135])

# Stewart
# d = np.array([0.444635, 0.535328, 0.154329])
# a = np.array([0.168856, 0.623913, 3.42525])

In [3]:
# distancia interatómica de 1.401 u.a.
RA = np.array([0, 0, 0])
RB = np.array([1.401, 0, 0])

Se buscan los elementos de matriz $f_{pq}$ dados por la expresión siguiente:

$$ f_{pq} = \int \chi_p^* \left( -\frac{\nabla^2}{2} + \sum_C - \frac{Z_C}{r_C} \right) \chi_q \, \mathrm{d}\vec{\mathbf{x}} \,;
\quad
\vec{\mathbf{x}} = (\vec{\mathbf{r}}, \omega)
$$

La base de funciones a considerar es
$$ \chi_1 = \psi_+ \alpha \,, \quad
\chi_2 = \psi_+ \beta \,, \quad
\chi_3 = \psi_- \alpha \,, \quad
\chi_4 = \psi_- \beta \,,
$$

donde $\psi_\pm = c_\pm (\varphi_A \pm \varphi_B)$ con

$$ c_\pm = \frac{1}{\sqrt{2 (1 \pm S)}}\,, $$

la constante de normalización y

$$ \varphi_A = R_{1\mathrm{s}}^{k}(\vec{\mathbf{d}}, \vec{\mathbf{\alpha}}, r_A) Y_{lm}(\theta,\phi) \,,
\quad
\varphi_B = R_{1\mathrm{s}}^{k}(\vec{\mathbf{d}}, \vec{\mathbf{\beta}}, r_B) Y_{lm}(\theta,\phi)
$$

las funciones espaciales centradas en el núcleo $A$ y $B$, respectivamente. En esto último $R_{1\mathrm{s}}^{k}$ no es más que una combinación lineal de $k$ funciones Gaussianas $1\mathrm{s}$, también conocido como una $k$-contracción de Gaussianas $1\mathrm{s}$:

$$ R_{1\mathrm{s}}^{k}(\vec{\mathbf{d}}, \vec{\mathbf{\alpha}}, r) = \sum_{\nu=1}^{k} d_\nu R_{0}^{\mathrm{GTO}} (\alpha_\nu, r) \,;
\qquad \qquad
R_0^{\mathrm{GTO}} (\alpha, r) = \frac{2(2\alpha)^{3/4}}{\pi^{1/4}} \exp(-\alpha \, r^2) \,.
$$

La función $\varphi_A$ ($\varphi_B$) es entonces de la forma:

$$ \varphi_A = \sum_{\nu = 1}^{k} d_\nu \left( \frac{2\alpha_\nu}{\pi} \right)^{3/4} \exp(-\alpha_\nu \, r_A^2) \,.$$

Para simplificar notación considere

$$ G (\vec{\mathbf{r}}, \alpha_\nu, \vec{A}) = \left( \frac{2\alpha_\nu}{\pi} \right)^{3/4} g(\vec{\mathbf{r}}, \alpha_\nu, \vec{A})
\,; \qquad \qquad
g(\vec{\mathbf{r}}, \alpha_\nu, \vec{A}) = \exp(-\alpha_\nu \, r_A^2) \,.$$

En cuyo caso, la función $\varphi_A$ será reescrita como

$$ \varphi_A
= G_{\mu} (\vec{\mathbf{r}}, \vec{\mathbf{d}}, \vec{\mathbf{\alpha}}, \vec{A})
= \sum_{\nu = 1}^{k} d_{\nu \mu} G (\vec{\mathbf{r}}, \alpha_{\nu \mu}, \vec{A})
= \sum_{\nu = 1}^{k} d_{\nu \mu} \left( \frac{2\alpha_{\nu \mu}}{\pi} \right)^{3/4} \exp(-\alpha_{\nu \mu} \, r_A^2) \,.
$$

In [4]:
def GaussNorm(a:int):
    """
        Factor de normalización Gaussiano 1s
    a : exponente Gaussiano
    """
    return np.power(2*a/np.pi, 3/4)

Los coeficientes de contracción $d_{\mu \, \nu}$ y los coeficientes Gaussianos $\alpha_\nu$ fueron calculados previamente en el cuaderno de Jupyter `(1) STO-3G.ipynb` .

<div class="alert alert-block alert-info">
    <strong>Integral de traslape, $S_{\mu \, \nu}$</strong>
</div>

Aquí $S_{\mu \, \nu} = \braket{\varphi_A|\varphi_B} = \braket{\varphi_B|\varphi_A}$ es la integral de traslape dada por

$$ \begin{align}
    S_{\mu \, \nu} 
    &= \int G_{\mu}^* (\vec{\mathbf{r}}, \vec{\mathbf{\alpha}}, \vec{A}) G_{\nu} (\vec{\mathbf{r}}, \vec{\mathbf{\beta}}, \vec{B}) \,\mathrm{d}V \\
    &= \int \sum_{p=1}^{k} d_{p \mu}^* \, G^* (\vec{\mathbf{r}}, \alpha_{p \mu}, \vec{A}) \,
        \sum_{q=1}^{k} d_{q \nu} \, G (\vec{\mathbf{r}}, \beta_{q \nu}, \vec{B}) \,\mathrm{d}V \\
    &= \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu} \int G (\vec{\mathbf{r}}, \alpha_{p \mu}, \vec{A}) \,
         G (\vec{\mathbf{r}}, \beta_{q \nu}, \vec{B}) \,\mathrm{d}V \\
    &= \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu} S_{pq}
\end{align}
$$

Esta última integral de acuerdo con Szabo y Ostlund es:

$$ \begin{align}
    S_{pq}
    &= \int G (\vec{\mathbf{r}}, \alpha_{p \mu}, \vec{A}) \, G (\vec{\mathbf{r}}, \beta_{q \nu}, \vec{B}) \,\mathrm{d}V \\
    &= \left( \frac{2\alpha_{p \mu}}{\pi} \right)^{3/4} \left( \frac{2\beta_{q \nu}}{\pi} \right)^{3/4}
        \int \exp(-\alpha_{p \mu} r^2_A) \exp(-\beta_{q \nu} \, r^2_B) \,\mathrm{d}V \\
    &= \left( \frac{2\alpha_{p \mu}}{\pi} \right)^{3/4} \left( \frac{2\beta_{q \nu}}{\pi} \right)^{3/4}
        \left( \frac{\pi}{\alpha_{p \mu} + \beta_{q \nu}} \right)^{3/2} \exp\left( -\frac{\alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}} |\mathbf{R}_A - \mathbf{R}_B|^2 \right)
\end{align}
$$
<br>

El término exponencial aparece a lo largo de las integrales por calcular, motivo por el cual considere las siguientes convenciones:

$$ K = \exp(\mathrm{arg})\,; \qquad \mathrm{arg} = -\frac{a b}{a + b} |\mathbf{R}_A - \mathbf{R}_B|^2$$

In [5]:
def arg(a:float, b:float, RA:np.array, RB:np.array):
    """
        Argumento del factor pre-exponencial K
    (a, b) : exponente orbital Gaussiano
    (RA, RB) : coordenada del núcleo (A, B)
    """
    p = a+b # exponente total
    mu = a*b/p
    RAB2 = np.square(np.linalg.norm(RA-RB))
    return -mu*RAB2

In [6]:
def K(a:float, b:float, RA:np.ndarray, RB:np.ndarray):
    """
        Factor pre-exponencial
    (a, b) : exponente orbital Gaussiano
    (RA, RB) : coordenada del núcleo (A, B)
    """
    return np.exp(arg(a, b, RA, RB))

In [7]:
def Spq(a:float, b:float, RA:np.ndarray, RB:np.ndarray):
    """
        Integral de traslape S_pq (normalizada)
    (a, b) : exponente orbital Gaussiano
    (RA, RB) : coordenada del núcleo (A, B)
    """
    return GaussNorm(a) * GaussNorm(b) * np.power(np.pi/(a+b), 3/2) * K(a, b, RA, RB)

La expresión final de la integral de traslape $S_{\mu \nu}$ es

$$ S_{\mu \nu} 
=  \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu}  \left( \frac{2\alpha_{p \mu}}{\pi} \right)^{3/4} \left( \frac{2\beta_{q \nu}}{\pi} \right)^{3/4}
        \left( \frac{\pi}{\alpha_{p \mu} + \beta_{q \nu}} \right)^{3/2} \exp\left( -\frac{\alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}} |\mathbf{R}_A - \mathbf{R}_B|^2 \right) $$

con $\mathbf{R}_A$ y $\mathbf{R}_B$ la posición del núcleo $A$ y $B$ en unidades atómicas.

In [8]:
def Smn(d:np.ndarray, a:np.ndarray, RA:np.ndarray, RB:np.ndarray):
    """
        Integral de traslape total S_mn
    d : vector de coeficientes de expansión (d1, d2, ..., dk)
    a : vector de exponentes orbitales Gaussianos (a1, a2, ..., ak)
    (RA, RB) : coordenada del núcleo (A, B)
    """
    Mmn = 0 # elemento de matriz
    k = len(d)
    for p in range(k):
        for q in range(k):
            Mmn += d[p] * d[q] * Spq(a[p], a[q], RA, RB) # elemento de matriz
    return Mmn

In [9]:
SAA = Smn(d, a, RA, RA) # elemento de matriz S_AA y S_BB
SAB = Smn(d, a, RA, RB) # elemento de matriz S_AB y S_BA

print("S_AA y S_BB:", SAA)
print("S_AB y S_BA:", SAB)

S_AA y S_BB: 1.0000002270615276
S_AB y S_BA: 0.6589699557825579


Los elementos de la diagonal no son más que la condición de normalización. Dado que es hasta el séptimo decimal de estos elementos que aparece un valor diferente de cero, todos los valores serán redondeados a 6 cifras de precisión decimal.

In [10]:
# cifra decimal a la cual se redondearán los elementos de matriz S
# la cifra será el magnitud superior del orden de magnitud del primer dígito diferente de cero para la condición de normalización.
vdecimal = abs(floor(log10(abs(float(SAA) % 1)))+1)

In [11]:
def truncar_float(valor:float, decimal:int):
    """ Remueve los decimales de 'valor' después de la posición 'decimal'
    """
    factor = 10 ** decimal
    return int(valor * factor) / factor

La matriz de traslape $S$ con una distncia interatómica de $1.401$ u.a. es de la forma

$$ \boxed{S
= \begin{pmatrix} S_{AA} & S_{AB} \\ S_{BA} & S_{BB} \end{pmatrix}
= \begin{pmatrix} 1.0 & 0.658969 \\ 0.658969 & 1.0 \end{pmatrix}}
$$

<div class="alert alert-block alert-info">
        <strong>Integral cinética, $T_{\mu \, \nu}$</strong>
</div>

La integral cinética está dada por

$$T_{\mu \, \nu}  = \braket{\varphi_A| -\frac{\nabla^2}{2} |\varphi_B} = \braket{\varphi_B| -\frac{\nabla^2}{2} |\varphi_A} \,.$$

Desarrollando,

$$ \begin{align}
    T_{\mu \, \nu} 
    &= \int G_{\mu}^* (\vec{\mathbf{r}}, \vec{\mathbf{\alpha}}, \vec{A}) \left( -\frac{\nabla^2}{2} \right)
        G_{\nu} (\vec{\mathbf{r}}, \vec{\mathbf{\beta}}, \vec{B}) \,\mathrm{d}V \\
    &= \int \sum_{p=1}^{k} d_{p \mu}^* \, G (\vec{\mathbf{r}}, \alpha_{p \mu}, \vec{A}) \,
        \left( -\frac{\nabla^2}{2} \right)
        \sum_{q=1}^{k} d_{q \nu} \, G (\vec{\mathbf{r}}, \beta_{q \nu}, \vec{B}) \,\mathrm{d}V \\
    &= \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu} \underbrace{\int G (\vec{\mathbf{r}}, \alpha_{p \mu}, \vec{A}) \,
        \left( -\frac{\nabla^2}{2} \right)
         G (\vec{\mathbf{r}}, \beta_{q \nu}, \vec{B}) \,\mathrm{d}V}_{T_{pq}}
\end{align}
$$

La expresión a resolver es 

$$ T_{\mu \, \nu} = \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu} \, T_{pq} \,.$$

Desarrollando para $T_{pq}$ se obtiene

$$ \begin{align}
    T_{pq}
    &= \int G (\vec{\mathbf{r}}, \alpha_{p \mu}, \vec{A}) \left( -\frac{\nabla^2}{2} \right) G (\vec{\mathbf{r}}, \beta_{q \nu}, \vec{B}) \,\mathrm{d}V \\
    &= \left( \frac{2\alpha_{p \mu}}{\pi} \right)^{3/4} \left( \frac{2\beta_{q \nu}}{\pi} \right)^{3/4}
        \underbrace{\int \exp(-\alpha_{p \mu} r^2_A) \left( -\frac{\nabla^2}{2} \right) \exp(-\beta_{q \nu} \, r^2_B) \,\mathrm{d}V}_{I_1} \,.
\end{align}
$$

La integral $I_1$ de la expresión previa de acuerdo con Szabo y Ostlund es:

$$ I_1 = \frac{\alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}}
    \left[ 3 - \frac{2 \alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}} |\mathbf{R}_A - \mathbf{R}_B|^2 \right]
    \left( \frac{\pi}{\alpha_{p \mu} + \beta_{q \nu}} \right)^{3/2}
    \exp\left( -\frac{\alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}} |\mathbf{R}_A - \mathbf{R}_B|^2 \right) \,.$$

Observe que los últimos dos factores de $I_1$ multiplicados por los respectivos coeficientes de normalización no es más que $S_{pq}$.

In [12]:
def Tpq(a:np.ndarray, b:np.ndarray, RA:np.ndarray, RB:np.ndarray):
    """
        Integral cinética S_mn (normalizada)
    d : vector de coeficientes de expansión (d1, d2, ..., dk)
    a : vector de exponentes orbitales Gaussianos (a1, a2, ..., ak)
    (RA, RB) : coordenada del núcleo (A, B)
    """
    RAB2 = np.square(np.linalg.norm(RA-RB))
    return (a*b)/(a+b) * (3 - 2*(a*b)/(a+b)*RAB2 ) * Spq(a, b, RA, RB)

De lo cual, la expresión final de la integral cinética $T_{\mu \nu}$ no es más que:

$$ T_{\mu \nu} 
=  \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu}  \left( \frac{2\alpha_{p \mu}}{\pi} \right)^{3/4} \left( \frac{2\beta_{q \nu}}{\pi} \right)^{3/4}
        \frac{\alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}} \left[ 3 - \frac{2 \alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}} |\mathbf{R}_A - \mathbf{R}_B|^2 \right]
        \left( \frac{\pi}{\alpha_{p \mu} + \beta_{q \nu}} \right)^{3/2} \exp\left( -\frac{\alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}} |\mathbf{R}_A - \mathbf{R}_B|^2 \right) \,.
$$

con $\mathbf{R}_A$ y $\mathbf{R}_B$ la posición del núcleo $A$ y $B$ en unidades atómicas.

In [13]:
def Tmn(d:np.ndarray, a:np.ndarray, RA:np.ndarray, RB:np.ndarray):
    """
        Integral cinética total S_mn
    d : vector de coeficientes de expansión (d1, d2, ..., dk)
    a : vector de exponentes orbitales Gaussianos (a1, a2, ..., ak)
    (RA, RB) : coordenada del núcleo (A, B)
    """
    Mmn = 0 # elemento de matriz
    k = len(d)
    for p in range(k):
        for q in range(k):
            Mmn += d[p] * d[q] * Tpq(a[p], a[q], RA, RB) # elemento de matriz
    return Mmn

In [14]:
TAA = Tmn(d, a, RA, RA) # elemento de matriz T_AA y T_BB
TAB = Tmn(d, a, RA, RB) # elemento de matriz T_AB y T_BA

print("T_AA & T_BB:", TAA)
print("T_AB & T_BA:", TAB)

T_AA & T_BB: 0.7601270778257048
T_AB & T_BA: 0.2361332038713467


La matriz cinética $T$ es de la forma

$$ \boxed{T
= \begin{pmatrix} T_{AA} & T_{AB} \\ T_{BA} & T_{BB} \end{pmatrix}
= \begin{pmatrix} 0.7601270778257048 & 0.2361332038713467 \\ 0.2361332038713467 & 0.7601270778257048 \end{pmatrix}}
$$

<div class="alert alert-block alert-info">
        <strong>Integral Coulombiana del i-ésimo núcleo, $V^i_{\mu \, \nu}$</strong>
</div>

La integral Coulombiana del $i$-ésimo núcleo está dada por

$$V^i_{\mu \, \nu}  = \braket{\varphi_A| \sum_C  - \frac{Z_C}{r_c} |\varphi_B} = \braket{\varphi_B| \sum_C  -\frac{Z_C}{r_c} |\varphi_A} \,.$$

Desarrollando,

$$ \begin{align}
    V^i_{\mu \, \nu} 
    &= \int G_{\mu}^* (\vec{\mathbf{r}}, \vec{\mathbf{\alpha}}, \vec{A}) \left( \sum_C -\frac{Z_C}{r_c} \right)
        G_{\nu} (\vec{\mathbf{r}}, \vec{\mathbf{\beta}}, \vec{B}) \,\mathrm{d}V \\
    &= \int \sum_{p=1}^{k} d_{p \mu}^* \, G^* (\vec{\mathbf{r}}, \alpha_{p \mu}, \vec{A}) \,
        \left( \sum_C -\frac{Z_C}{r_c} \right)
        \sum_{q=1}^{k} d_{q \nu} \, G (\vec{\mathbf{r}}, \beta_{q \nu}, \vec{B}) \,\mathrm{d}V \\
    &= \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu} \int G^* (\vec{\mathbf{r}}, \alpha_{p \mu}, \vec{A}) \,
        \left( \sum_C -\frac{Z_C}{r_c} \right)
         G (\vec{\mathbf{r}}, \beta_{q \nu}, \vec{B}) \,\mathrm{d}V \\
    &= \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu} \sum_C \int G^* (\vec{\mathbf{r}}, \alpha_{p \mu}, \vec{A}) \,
        \left( -\frac{Z_C}{r_c} \right)
         G (\vec{\mathbf{r}}, \beta_{q \nu}, \vec{B}) \,\mathrm{d}V
\end{align}
$$

La expresión por evaluar es entonces

$$ V^i_{\mu \, \nu} = \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu} \sum_C V^i_{pq} (C) $$

Esta última integral de acuerdo con Szabo y Ostlund es:

$$ \begin{align}
    V^i_{pq} (C)
    &= \int G^* (\vec{\mathbf{r}}, \alpha_{p \mu}, \vec{A}) \left( -\frac{Z_C}{r_c} \right) G (\vec{\mathbf{r}}, \beta_{q \nu}, \vec{B}) \,\mathrm{d}V \\
    &= \left( \frac{2\alpha_{p \mu}}{\pi} \right)^{3/4} \left( \frac{2\beta_{q \nu}}{\pi} \right)^{3/4}
        \int \exp(-\alpha_{p \mu} r^2_A) \left( -\frac{Z_C}{r_c} \right) \exp(-\beta_{q \nu} \, r^2_B) \,\mathrm{d}V \\
    &= \left( \frac{2\alpha_{p \mu}}{\pi} \right)^{3/4} \left( \frac{2\beta_{q \nu}}{\pi} \right)^{3/4}
        \left( -\frac{2\pi}{\alpha_{p \mu} + \beta_{q \nu}} \right) Z_C \,
        \exp\left( -\frac{\alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}} |\mathbf{R}_A - \mathbf{R}_B|^2 \right)
        F_0 \left[ (\alpha_{p \mu} + \beta_{q \nu}) |\mathbf{R}_P - \mathbf{R}_C|^2 \right]
\end{align}
$$

donde $F_0(t)$ es

$$ F_0(t) = \frac{1}{2} \left( \frac{\pi}{t} \right)^{1/2} \mathrm{erf}(t^{1/2}) \,;
\quad
\text{con} \quad \mathrm{erf}(z) = \frac{2}{\sqrt{\pi}} \int_0^{z} e^{-t^2} \, \mathrm{d}t
$$

y $\mathbf{R}_P$ es

$$ \mathbf{R}_P = \frac{\alpha \mathbf{R}_A + \beta \, \mathbf{R}_B}{\alpha + \beta} \,.$$

In [15]:
def F0(t:float):
    """
        Función de Boys, n=0
    t : argumento de la función
    """
    return (1/2) * np.sqrt(np.pi/t) * erf(np.sqrt(t))

In [16]:
def RP(a:float, b:float, RA:np.ndarray, RB:np.ndarray):
    """
        Coordenada de centro de carga
    (a, b) : exponente orbital Gaussiano
    (RA, RB) : coordenada del núcleo (A, B)
    """
    p = a + b # exponente total
    return (a*RA+b*RB)/p

<div class="alert alert-block alert-info">
        <strong>Caso particular.</strong> Todos los núcleos son iguales.
</div>

$$\begin{align}
    \int \exp(-\alpha \, r^2_A) \left( -\frac{Z_A}{r_A} \right) \exp(-\beta \, r^2_A) \,\mathrm{d}V
    &= -Z_A \int \frac{\exp[ - (\alpha + \beta) \, r^2]}{r} \,\mathrm{d}V \\
    &= -Z_A \int \mathrm{d}\Omega \int r \exp[ - (\alpha + \beta) \, r^2] \,\mathrm{d}r \\
    &= - Z_A (4 \pi) \frac{1}{2(\alpha + \beta)} \\
    &= - \frac{2 \pi Z_A}{\alpha + \beta}
\end{align}
$$

In [17]:
def Vpq_AB(a:float, b:float, RA:np.ndarray, RB:np.ndarray, RC:np.ndarray, ZC:float):
    """
    (a, b) : exponente orbital Gaussiano
    (RA, RB, RC) : coordenada del núcleo (A, B, C)
    ZC : carga del núcleo C
    """
    factor = -2*np.pi/(a+b) * ZC # factor común del término pq
    
    if np.array_equal(RA, RB) and np.array_equal(RB, RC): # todos los núcleos iguales
        Vpq = factor
    else: # cualquier otro caso
        RAB2 = np.square(np.linalg.norm(RA-RB))
        Rp = RP(a, b, RA, RB) # coordenada de centro de carga
        RPC2 = np.square(np.linalg.norm(Rp-RC))
        Vpq = factor * K(a, b, RA, RB) * F0((a+b)*RPC2)
    return Vpq

La expresión final para el elemento de matriz $V^i_{\mu \, \nu}$ es

$$ V^i_{\mu \, \nu}
= \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu} \left[ V^i_{pq}(A) + V^i_{pq}(B) \right]
= \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu} V^i_{pq}(A)
    + \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu}  V^i_{pq}(B)
$$

In [18]:
def Vmn(d:np.ndarray, a:np.ndarray, RA:np.ndarray, RB:np.ndarray, RC:np.ndarray, ZC:float):
    """
        Integral cinética S_mn
    d : vector de coeficientes de expansión (d1, d2, ..., dk)
    a : vector de exponentes orbitales Gaussianos (a1, a2, ..., ak)
    (RA, RB, RC) : coordenada del núcleo (A, B, C)
    ZC : carga del núcleo C
    """
    Mmn1 = 0 # elemento de matriz V^i_AA
    Mmn2 = 0 # elemento de matriz V^i_AB y V^i_BA
    Mmn3 = 0 # elemento de matriz V^i_BB
    k = len(d)

    for p in range(k):
        for q in range(k):
            factor = d[p] * d[q] * GaussNorm(a[p]) * GaussNorm(a[q]) # factor común del término p, q
            Mmn1 += factor * Vpq_AB(a[p], a[q], RA, RA, RC, ZC)
            Mmn2 += factor * Vpq_AB(a[p], a[q], RA, RB, RC, ZC)
            Mmn3 += factor * Vpq_AB(a[p], a[q], RB, RB, RC, ZC)
    
    return [Mmn1, Mmn2, Mmn3] # elementos de matriz (V^i_AA, V^i_AB, V^i_BB)

In [19]:
ZA, ZB = 1, 1

V1 = Vmn(d, a, RA, RB, RA, ZA) # matriz V^1 (núcleo A)
V2 = Vmn(d, a, RA, RB, RB, ZB) # matriz V^2 (núcleo B)

print("V1:", V1)
print("V2:", V2)

V1: [np.float64(-1.2266836047073328), np.float64(-0.5969507766237366), np.float64(-0.653481099086147)]
V2: [np.float64(-0.653481099086147), np.float64(-0.5969507766237366), np.float64(-1.2266836047073328)]


La matriz Coulombiana $V^1$ y $V^2$ son entonces

$$ \boxed{V^1
= \begin{pmatrix} V^1_{AA} & V^1_{AB} \\ V^1_{BA} & V^1_{BB} \end{pmatrix}
= \begin{pmatrix} -1.2266836047073328 & -0.5969507766237366 \\ -0.5969507766237366 & -0.653481099086147 \end{pmatrix}}
$$

$$ \boxed{V^2
= \begin{pmatrix} V^2_{AA} & V^2_{AB} \\ V^2_{BA} & V^2_{BB} \end{pmatrix}
= \begin{pmatrix} -0.653481099086147 & -0.5969507766237366 \\ -0.5969507766237366 & -1.2266836047073328 \end{pmatrix}}
$$

<div class="alert alert-block alert-info">
<strong>Coeficiente de normalización, $c_{\pm}$</strong>
</div>

In [20]:
def cPM(d:np.array, a:np.array, RA:np.array, RB:np.array, signo:str):
    """
        Constante de normalización c+-
            Psi = c+- (Phi_A +- Phi_B)
    d : vector de coeficientes de expansión (d1, d2, ..., dk)
    a : vector de exponentes orbitales Gaussianos (a1, a2, ..., ak)
    (RA, RB) : coordenada del núcleo (A, B)
    signo : signo de S en la normalización
         1 -> positivo
        -1 -> negativo
    """
    S = truncar_float(Smn(d, a, RA, RB), vdecimal)
    if signo == 'plus':
        return 1/np.sqrt(2*(1+S))
    else:
        return 1/np.sqrt(2*(1-S))

<div class="alert alert-block alert-info">
<strong>Elemento de matriz $f_{pp}$</strong>
</div>

$$ f_{pp}
    = |c_\pm|^2 \left[ 2 ( T_{AA} \pm_p T_{AB} )
    + (V^1_{AA} + V^1_{BB}) + (V^2_{AA} + V^2_{BB})
    \pm_p 2 ( V^1_{AB} + V^2_{AB} ) \right]
$$

In [21]:
def fpp(p:int, d:np.ndarray, a:np.ndarray, RA:np.ndarray, RB:np.ndarray, ZA:float, ZB:float):
    """
        Elemento de matriz f_pp
    p : elemento de la base, base = {X_1, X_2, X_3, X_4}
        p = 1, 2, 3, 4
    d : vector de coeficientes de expansión (d1, d2, ..., dk)
    a : vector de exponentes orbitales Gaussianos (a1, a2, ..., ak)
    (RA, RB) : coordenada del núcleo (A, B)
    (ZA, ZB) : carga del núcleo (A, B)
    """
    TAA = Tmn(d, a, RA, RA) # elemento de matriz T_AA y T_BB
    TAB = Tmn(d, a, RA, RB) # elemento de matriz T_AB y T_BA
    V1_AA, V1_AB, V1_BB = Vmn(d, a, RA, RB, RA, ZA) # elementos de matriz V^1_AA, V^1_AB y V^1_BB
    V2_AA, V2_AB, V2_BB = Vmn(d, a, RA, RB, RB, ZB) # elementos de matriz V^2_AA, V^2_AB y V^2_BB

    if p in [1,2]:
        c2 = np.square(cPM(d, a, RA, RB, 'plus'))
        term = c2*( 2*(TAA + TAB) + (V1_AA + V1_BB) + (V2_AA + V2_BB) + 2*(V1_AB + V2_AB) )
    else:
        c2 = np.square(cPM(d, a, RA, RB, 'minus'))
        term = c2*( 2*(TAA - TAB) + (V1_AA + V1_BB) + (V2_AA + V2_BB) - 2*(V1_AB + V2_AB) )
    return term

In [22]:
# Segura
d = np.array([0.1545314772435472, 0.5354555247125108, 0.4443376735387033])
a = np.array([3.423766464751332, 0.6234266839478166, 0.1687830644383135])

f11 = fpp(1, d, a, RA, RB, ZA, ZB)
f33 = fpp(3, d, a, RA, RB, ZA, ZB)

print("f11 & f22:", f11)
print("f33 & f44:", f33)

f11 & f22: -1.252468234996496
f33 & f44: -0.47581972486855656


In [23]:
# Stewart
d = np.array([0.444635, 0.535328, 0.154329])
a = np.array([0.168856, 0.623913, 3.42525])

f11 = fpp(1, d, a, RA, RB, ZA, ZB)
f33 = fpp(3, d, a, RA, RB, ZA, ZB)

print("f11 & f22:", f11)
print("f33 & f44:", f33)

f11 & f22: -1.2524790585335392
f33 & f44: -0.4759356598814759
